In [4]:
import pandas as pd
import ast  # For safely evaluating string-formatted lists
import json # For handling nested JSON
import sys  # For error checking
from google.colab import files # Import Colab's file handling

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
order_path = '/content/drive/MyDrive/WWT Dataset/order_data.csv'
customer_path = '/content/drive/MyDrive/WWT Dataset/customer_data.csv'
store_path = '/content/drive/MyDrive/WWT Dataset/store_data.csv'

try:
    print("Loading datasets from Google Drive...")
    order_data = pd.read_csv(order_path)
    customer_data = pd.read_csv(customer_path)
    store_data = pd.read_csv(store_path)

    print("All files loaded successfully.")
    print(f"Loaded {len(order_data)} orders.")

except FileNotFoundError as e:
    print(f"--- ERROR ---")
    print(f"File not found. Did you paste the correct path?")
    print(f"Path I tried to use: {e.filename}")
    sys.exit()

Loading datasets from Google Drive...
All files loaded successfully.
Loaded 1048575 orders.


In [ ]:
# --- This is the main processing script  ---

import pandas as pd
import ast
import json
import sys
import numpy as np # Import numpy for keyword search

print("--- Starting Data Preparation   ---")

# Check if 'order_data' exists from Cell 2
if 'order_data' not in locals():
    print("ERROR: 'order_data' not found. Please re-run Cell 2 to load the data.")
else:
    try:
        # --- 2. CLEAN & TRANSFORM (The Fast Way) ---
        print(f"Original 'order_data' rows: {len(order_data)}")
        print("Transforming 'ORDERS' column...")

        print("Step 2a (Filling NaNs and ast.literal_eval)...")
        order_data['ORDERS'] = order_data['ORDERS'].fillna('{"orders": []}')
        order_data['ORDERS_dict'] = order_data['ORDERS'].apply(ast.literal_eval)
        print("Step 2a: COMPLETE.")

        print("Step 2b (Normalizing with record_path)...")
        meta_cols = [
            'CUSTOMER_ID', 'STORE_NUMBER', 'ORDER_CREATED_DATE',
            'ORDER_ID', 'ORDER_CHANNEL_NAME',
            'ORDER_SUBCHANNEL_NAME', 'ORDER_OCCASION_NAME'
        ]

        final_data = pd.json_normalize(
            order_data.to_dict('records'),
            record_path=['ORDERS_dict', 'orders', 'item_details'],
            meta=meta_cols,
            errors='ignore'
        )
        print(f"Step 2b: COMPLETE. New expanded row count is: {len(final_data)}")

        # --- !! ADVANCED FILTERING (Keywords and Price) !! ---
        print(f"Filtering out non-product 'noise' rows...")
        rows_before = len(final_data)

        # 1. Filter by keywords (Your list)
        non_product_keywords = ['memo', 'blankline', 'asap', 'paid', 'not paid', 'tax', 'fee']
        keyword_mask = final_data['item_name'].str.lower().str.contains(
            '|'.join(non_product_keywords),
            na=False
        )
        final_data = final_data[~keyword_mask]
        rows_after_keywords = len(final_data)
        print(f"Removed {rows_before - rows_after_keywords} keyword 'noise' rows.")


        # 2. Filter by zero price (Your new logic)
        print("Filtering out zero-price items...")
        final_data = final_data[final_data['item_price'] > 0]
        rows_after_price = len(final_data)
        print(f"Removed {rows_after_keywords - rows_after_price} zero-price rows.")

        print(f"Filtering: COMPLETE. Total rows removed: {rows_before - rows_after_price}")
        # --- End of filtering step ---

        # --- 3. MERGE with Customer and Store Data ---
        print("Step 3 (Merging customer/store data)...")
        final_data = pd.merge(final_data, customer_data, on='CUSTOMER_ID', how='left')
        final_data = pd.merge(final_data, store_data, on='STORE_NUMBER', how='left')
        print("Step 3: COMPLETE. All data merged.")

        # --- 4. FEATURE ENGINEERING & CLEANUP ---
        print("Step 4 (Adding Date features)...")
        final_data['ORDER_CREATED_DATE'] = pd.to_datetime(final_data['ORDER_CREATED_DATE'])
        final_data['Order_Day_of_Week'] = final_data['ORDER_CREATED_DATE'].dt.day_name()
        final_data['Order_Month'] = final_data['ORDER_CREATED_DATE'].dt.month_name()
        final_data['Order_Hour'] = final_data['ORDER_CREATED_DATE'].dt.hour
        final_data['Order_Date'] = final_data['ORDER_CREATED_DATE'].dt.date
        print("Step 4: COMPLETE.")

        # --- 5. EXPORT TO CSV ---
        output_filename = 'wings_r_us_powerbi_master_FINAL_CLEAN.csv'
        final_data.to_csv(output_filename, index=False)

        print("\n--- 🥳 SUCCESS! ---")
        print(f"Your file is ready: {output_filename}")
        print(f"Total Rows in new file: {len(final_data)}")
        print("This file is fully cleaned and ready for Power BI.")
        print("You can now run Cell 4 to download this new file.")

    except Exception as e:
        print("\n--- 😭 ERROR ---")
        print("The script failed. Here is the error message:")
        print(e)

--- Starting Data Preparation   ---
Original 'order_data' rows: 1048575
Transforming 'ORDERS' column...
Step 2a (Filling NaNs and ast.literal_eval)...
Step 2a: COMPLETE.
Step 2b (Normalizing with record_path)...
Step 2b: COMPLETE. New expanded row count is: 4180341
Filtering out non-product 'noise' rows...
Removed 2096030 keyword 'noise' rows.
Filtering out zero-price items...
Removed 6078 zero-price rows.
Filtering: COMPLETE. Total rows removed: 2102108
Step 3 (Merging customer/store data)...
Step 3: COMPLETE. All data merged.
Step 4 (Adding Date features)...


/tmp/ipython-input-67415164.py:72: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_data['ORDER_CREATED_DATE'] = pd.to_datetime(final_data['ORDER_CREATED_DATE'])


Step 4: COMPLETE.
